# LLM 개발 환경설정



---



- 💡**참고 교재**
    - 핸즈온LLM https://www.hanbit.co.kr/store/books/look.php?p_code=B2599445240
- 💡 **NOTE**
    - 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.

---

#파이썬 버전 확인

- Colab

In [ ]:
!python --version

- (Local) win/mac

In [ ]:
py --version

In [ ]:
py --list   # 설치된 모든 파이썬 버전 확인

In [ ]:
py -3.12    # 특정 버전 실행 : Python 3.12 실행

In [ ]:
import sys
if sys.version_info < (3, 9):
    print("Python 3.9 미만 버전이네요. 업그레이드 필요!")
else:
    print("최신 버전이네요!")


In [ ]:
import platform
print(f"현재 파이썬 버전은 🚀 {platform.python_version()} 🚀 입니다")




---



# 대표 라이브러리 설치


## **transformers**


- **Hugging Face가 개발한 오픈소스 라이브러리(Transformer 모델 라이브러리)**
- 자연어 처리, 컴퓨터 비전, 음성 등 다양한 분야의 최신 Transformer 기반 사전 학습된(pretrained) 모델을 쉽고 빠르게 사용할 수 있도록 지원함
- 3개의 딥러닝 라이브러리 지원 :  PyTorch, TensorFlow, Jax
- 상세설명 : https://pypi.org/project/transformers/



- **지원 플랫폼 / 요구사항**
    - Python 3.9 이상, PyTorch 2.1+, TensorFlow 2.6+, Flax 0.4.1+
- **용도 / 특징**
    - 텍스트, 이미지, 오디오, 멀티모달 모델들처럼 다양한 영역의 pretrained 모델을 간단한 API(pipeline)로 빠르게 사용 가능
- **추천 사용 케이스**
    - 빠르고 쉽게 NLP, 컴퓨터 비전, 음성 등의 사전학습 모델을 사용하거나 fine-tuning 하고자 할 때

- **설치 방법**

In [ ]:
# 설치된 transformers 버전 확인  -->(ex: Version: 4.55.4)
!pip show transformers | grep Version

In [ ]:
# Phi-3 모델과 호환성 때문에 transformers 4.48.3 버전을 사용합니다.
!pip install transformers==4.48.3

In [ ]:
# 설치된 PyTorch 버전 확인  -->(ex: Version: 2.8.0+cu126 (+cu126-->CUDA 12.6 버전 지원))
!pip show torch | grep Version

# pip install torch==2.8.0

### **QuickTour**

#### **감정분석**

In [ ]:
# 감정 분석 파이프라인
from transformers import pipeline

classifier = pipeline('sentiment-analysis')
print(classifier('We are very happy to introduce pipeline to the transformers repository.'))

In [ ]:
from transformers import pipeline

nlp = pipeline("sentiment-analysis")
print(nlp("이 강의는 정말 재미있어요!"))


#### **이미지 객체 감지**

In [ ]:
import requests
from PIL import Image, ImageDraw
from transformers import pipeline

# 이미지 다운로드
url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/coco_sample.png"
image_data = requests.get(url, stream=True).raw
image = Image.open(image_data)

# 객체 감지 파이프라인
object_detector = pipeline("object-detection")
results = object_detector(image)

# 원본 이미지에 바운딩 박스 그리기
draw = ImageDraw.Draw(image)

for result in results:
    box = result["box"]
    label = result["label"]
    score = result["score"]

    # 박스 좌표
    x1, y1, x2, y2 = box["xmin"], box["ymin"], box["xmax"], box["ymax"]

    # 박스 그리기
    draw.rectangle(((x1, y1), (x2, y2)), outline="red", width=3)
    # 레이블 + 점수 표시
    draw.text((x1, y1 - 10), f"{label} {score:.2f}", fill="yellow")

# 결과 출력 (Colab 환경에서 시각화)
image.show()   # 로컬 환경이면 새 창
display(image) # Colab/Jupyter 환경이면 노트북 안에 표시


#### **기계 번역**
- 💡주의: 모델 다운로드시 오래 걸릴 수 있다.

- Access Token 없이 공개 모델 다운로드


In [ ]:
# 인증이 필요하지 않은 대체 모델들
from transformers import pipeline

# 옵션 1: Facebook의 NLLB 모델 (추천)
translator = pipeline(
    'translation',
    model='facebook/nllb-200-distilled-600M',
    src_lang='eng_Latn',
    tgt_lang='kor_Hang',
    max_length=512
)

text = "Hello, how are you today?"
result = translator(text)
print(f"원문: {text}")
print(f"번역: {result[0]['translation_text']}")

In [ ]:
import builtins
input = builtins.input  # input 함수를 원래대로 복구

while True:
    text = input('한글로 번역할 문장 입력("q"): ')
    if (text == 'q') :
        break
    result = translator(text)
    print(f"원문: {text}")
    print(f"번역: {result[0]['translation_text']}")
    print( )
print('번역 종료!')

- Access Token 넣고 비공개 모델 접근

In [ ]:
# 1단계: Hugging Face 토큰 생성 및 설정
from huggingface_hub import login
import os

# Google Colab에서 토큰 입력 (한 번만 실행)
# https://huggingface.co/settings/tokens 에서 토큰 생성 후 입력
login()

# 또는 직접 토큰 입력 (보안상 권장하지 않음)
#os.environ["HUGGINGFACE_HUB_TOKEN"] = ""

In [ ]:
from transformers import pipeline

print("=== Helsinki-NLP 영한 번역 모델 로딩 ===")

# 올바른 영어→한국어 번역 모델
translator = pipeline('translation', model='Helsinki-NLP/opus-mt-tc-big-en-ko')

print("✅ 모델 로딩 완료!")

# 번역 실행
text = "Hello, how are you today?"
result = translator(text)
print(f"원문: {text}")
print(f"번역: {result[0]['translation_text']}")

- 💡[주의] 실행속도 느림(전체 실행 5분정도 걸림)

In [ ]:
from transformers import pipeline
import time
import re

class HighQualityTranslator:
    """고품질 양방향 번역기"""

    def __init__(self):
        self.translator = None
        self.model_info = {}
        self.translation_count = 0
        self._initialize_best_model()

    def _initialize_best_model(self):
        """최고 성능 모델 초기화"""

        print("🚀 고품질 번역기 초기화 중...")

        # 실제 작동하는 고품질 모델들 (우선순위 순)
        quality_models = [
            {
                'name': 'facebook/m2m100_1.2B',
                'description': 'Facebook M2M100 대형 모델 (최고 품질)',
                'size': '1.2GB',
                'type': 'm2m100_large'
            },
            {
                'name': 'facebook/m2m100_418M',
                'description': 'Facebook M2M100 중형 모델 (균형)',
                'size': '418MB',
                'type': 'm2m100'
            },
            {
                'name': 'facebook/nllb-200-distilled-600M',
                'description': 'Facebook NLLB-200 축약 모델',
                'size': '600MB',
                'type': 'nllb'
            }
        ]

        for model_config in quality_models:
            try:
                print(f"🔄 시도: {model_config['description']}")

                start_time = time.time()

                if 'm2m100' in model_config['type']:
                    # M2M100 모델 (가장 안정적)
                    self.translator = pipeline(
                        'translation',
                        model=model_config['name'],
                        device=-1,
                        torch_dtype='auto'
                    )
                    self.src_lang_en = 'en'
                    self.tgt_lang_ko = 'ko'
                    self.src_lang_ko = 'ko'
                    self.tgt_lang_en = 'en'

                elif model_config['type'] == 'nllb':
                    # NLLB 모델
                    self.translator = pipeline(
                        'translation',
                        model=model_config['name'],
                        device=-1
                    )
                    self.src_lang_en = 'eng_Latn'
                    self.tgt_lang_ko = 'kor_Hang'
                    self.src_lang_ko = 'kor_Hang'
                    self.tgt_lang_en = 'eng_Latn'

                load_time = time.time() - start_time

                # 테스트 번역으로 품질 확인
                test_en = "Hello, how are you?"
                test_ko = "안녕하세요, 어떻게 지내세요?"

                # 영→한 테스트
                en_result = self._translate_with_params(test_en, self.src_lang_en, self.tgt_lang_ko)
                # 한→영 테스트
                ko_result = self._translate_with_params(test_ko, self.src_lang_ko, self.tgt_lang_en)

                print(f"✅ 성공! 로딩 시간: {load_time:.2f}초")
                print(f"📊 영→한 테스트: '{test_en}' → '{en_result}'")
                print(f"📊 한→영 테스트: '{test_ko}' → '{ko_result}'")

                self.model_info = {
                    'name': model_config['name'],
                    'description': model_config['description'],
                    'size': model_config['size'],
                    'type': model_config['type'],
                    'load_time': f"{load_time:.2f}초"
                }

                break

            except Exception as e:
                print(f"❌ 실패: {str(e)[:60]}...")
                continue

        if self.translator is None:
            raise RuntimeError("사용 가능한 고품질 번역 모델을 찾을 수 없습니다.")

    def _translate_with_params(self, text, src_lang, tgt_lang):
        """파라미터를 사용한 번역 실행"""
        try:
            if 'm2m100' in self.model_info['type']:
                # M2M100 모델용
                result = self.translator(text, src_lang=src_lang, tgt_lang=tgt_lang, max_length=128)
            else:
                # NLLB 모델용
                result = self.translator(text, src_lang=src_lang, tgt_lang=tgt_lang, max_length=128)

            return result[0]['translation_text']

        except Exception as e:
            return f"번역 오류: {str(e)[:50]}..."

    def detect_language(self, text):
        """개선된 언어 감지"""
        # 한글 문자 비율 계산
        korean_chars = len(re.findall(r'[가-힣]', text))
        total_chars = len(re.sub(r'\s+', '', text))  # 공백 제외

        if total_chars == 0:
            return 'unknown'

        korean_ratio = korean_chars / total_chars

        # 한글이 30% 이상이면 한국어로 판단
        if korean_ratio >= 0.3:
            return 'ko'
        # 영문자나 숫자가 주를 이루면 영어로 판단
        elif re.search(r'[a-zA-Z]', text):
            return 'en'
        else:
            return 'unknown'

    def translate_en_to_ko(self, english_text):
        """영어→한국어 번역"""
        if not english_text.strip():
            return "번역할 텍스트가 없습니다."

        try:
            result = self._translate_with_params(
                english_text.strip(),
                self.src_lang_en,
                self.tgt_lang_ko
            )
            self.translation_count += 1
            return result

        except Exception as e:
            return f"영→한 번역 실패: {str(e)}"

    def translate_ko_to_en(self, korean_text):
        """한국어→영어 번역"""
        if not korean_text.strip():
            return "번역할 텍스트가 없습니다."

        try:
            result = self._translate_with_params(
                korean_text.strip(),
                self.src_lang_ko,
                self.tgt_lang_en
            )
            self.translation_count += 1
            return result

        except Exception as e:
            return f"한→영 번역 실패: {str(e)}"

    def auto_translate(self, text):
        """자동 언어 감지 및 번역"""
        if not text.strip():
            return "번역할 텍스트가 없습니다."

        detected_lang = self.detect_language(text)

        if detected_lang == 'ko':
            translation = self.translate_ko_to_en(text)
            return f"🇰🇷→🇺🇸 {translation}"
        elif detected_lang == 'en':
            translation = self.translate_en_to_ko(text)
            return f"🇺🇸→🇰🇷 {translation}"
        else:
            # 일단 영어로 가정하고 번역 시도
            translation = self.translate_en_to_ko(text)
            return f"🔤→🇰🇷 {translation}"

    def get_model_info(self):
        """모델 정보 반환"""
        info = self.model_info.copy()
        info['total_translations'] = self.translation_count
        return info

    def batch_translate(self, texts, direction='auto'):
        """배치 번역"""
        results = []

        print(f"📦 배치 번역 시작 ({len(texts)}개 문장)")

        for i, text in enumerate(texts, 1):
            if direction == 'en_to_ko':
                result = self.translate_en_to_ko(text)
            elif direction == 'ko_to_en':
                result = self.translate_ko_to_en(text)
            else:
                result = self.auto_translate(text)

            results.append(result)
            print(f"진행률: {i}/{len(texts)} ({i/len(texts)*100:.1f}%)")

        return results

# 실제 사용 및 테스트
print("=== 고품질 양방향 번역기 테스트 ===")

try:
    # 번역기 초기화
    translator = HighQualityTranslator()

    # 모델 정보 출력
    info = translator.get_model_info()
    print(f"\n📋 로딩된 모델 정보:")
    for key, value in info.items():
        print(f"   {key}: {value}")

    print(f"\n" + "="*60)

    # 1. 영어→한국어 번역 테스트
    print(f"🔤 영어→한국어 번역 테스트:")
    en_sentences = [
        "Good morning! How are you today?",
        "I love studying artificial intelligence and machine learning.",
        "The weather is beautiful, perfect for a walk in the park.",
        "Thank you so much for your help with this project.",
        "Python is an excellent programming language for beginners."
    ]

    for i, sentence in enumerate(en_sentences, 1):
        translation = translator.translate_en_to_ko(sentence)
        print(f"\n{i}. EN: {sentence}")
        print(f"   KO: {translation}")

    print(f"\n" + "="*60)

    # 2. 한국어→영어 번역 테스트
    print(f"🔤 한국어→영어 번역 테스트:")
    ko_sentences = [
        "안녕하세요! 오늘 기분이 어떠세요?",
        "저는 인공지능과 머신러닝을 공부하는 것을 좋아합니다.",
        "날씨가 정말 좋네요, 공원을 산책하기 완벽해요.",
        "이 프로젝트를 도와주셔서 정말 감사합니다.",
        "파이썬은 초보자들에게 훌륭한 프로그래밍 언어입니다."
    ]

    for i, sentence in enumerate(ko_sentences, 1):
        translation = translator.translate_ko_to_en(sentence)
        print(f"\n{i}. KO: {sentence}")
        print(f"   EN: {translation}")

    print(f"\n" + "="*60)

    # 3. 자동 언어 감지 번역 테스트
    print(f"🤖 자동 언어 감지 번역 테스트:")
    mixed_sentences = [
        "Hello world!",
        "안녕 세상!",
        "I'm learning Korean language.",
        "한국어를 배우고 있어요.",
        "Artificial Intelligence is amazing!",
        "인공지능은 정말 놀라워요!"
    ]

    for i, sentence in enumerate(mixed_sentences, 1):
        detected_lang = translator.detect_language(sentence)
        translation = translator.auto_translate(sentence)
        print(f"\n{i}. 입력: {sentence}")
        print(f"   감지언어: {detected_lang}")
        print(f"   번역: {translation}")

    # 4. 최종 통계
    final_info = translator.get_model_info()
    print(f"\n📊 번역 통계:")
    print(f"   총 번역 횟수: {final_info['total_translations']}회")
    print(f"   사용 모델: {final_info['description']}")

except Exception as e:
    print(f"❌ 번역기 오류: {e}")
    import traceback
    traceback.print_exc()



---



## **Phi‑3**

- **Microsoft가 개발한 소형 언어 모델(SLM) 시리즈**
    - Phi‑3‑mini (3.8B 파라미터), Phi‑3‑small (약 7B), Phi‑3‑medium (14B) 등 다양한 변형을 포함
- 파라미터 & 컨텍스트 윈도우
    - Phi-3-mini: 3.8B 파라미터, 4K 및 확장된 128K 컨텍스트 윈도우 지원
    - Phi‑3‑small: 약 7B, 기본 8K 컨텍스트
    - Phi‑3‑medium: 14B 기능, 더 넓은 컨텍스트 처리 능력
    - **microsoft/Phi-3-mini-4k-instruct**

- 학습 데이터 & 미세조정
    - 총 3.3조 토큰 이상으로 구성된 고품질 필터링 웹 데이터, 합성 데이터 등을 활용. 이후 **Supervised Fine-Tuning (SFT)** 과 Direct Preference Optimization (DPO) 기법을 통해 인간 선호도 및 안전 기준에 맞춰 미세 조정됨
- 성능 지표
    - 최소형 모델(Phi-3-mini)도 MMLU 69%, MT-bench 8.38 등 성능으로 Mixtral 8x7B, GPT-3.5와 유사한 수준
    - Phi‑3‑small 및 medium는 각각 MMLU 75% 및 78%, MT‑bench 8.7 및 8.9를 달성
    - 여러 벤치마크(HellaSwag, WinoGrande, TruthfulQA, HumanEval 등)에서 강력한 성능을 보임

- 관련 모델들
    - Phi-3-mini-128k-instruct: 128K 컨텍스트 지원 버전
    - Phi-3.5-mini-instruct: 다국어 지원 강화 버전 (2024년 8월)
    - Phi-4-mini-instruct: 최신 아키텍처 적용 버전



| 항목 | 세부 정보 |
|--- |--- |
|모델 크기 |3.8억 파라미터 |
|컨텍스트 길이 |4K 토큰 |
|훈련 토큰 수 |3.3조 토큰 |
|어휘 크기 |32,064 토큰 |
|아키텍처 |Dense decoder-only Transformer |
|출시일 |2024년 6월 |
|라이선스 |MIT 라이선스 |


|벤치마크 |Phi-3-mini 성능 |비교 모델 성능 |
|--- |--- |--- |
|MMLU |69% |GPT-3.5: ~70% |
|MT-bench |8.38 |GPT-3.5: ~8.5 |
|HumanEval |높은 성능 |확실하지 않음 |
|GSM-8K |우수 |확실하지 않음 |

### QuickTour

#### 모델 로드 및 토큰 생성


- 중간수준 API(Task-specific Models) 방식 --> 코랩(무료)에서 실행 오래 걸림(5분⭡)

In [ ]:
# Phi-3-mini 로드 및 간단 생성 (Transformers)
#   선형적 생성 흐름을 체험하며 모델 작동 원리를 설명하기 좋다.
from transformers import AutoModelForCausalLM, AutoTokenizer

# 모델과 토크나이저를 로드합니다.
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")


- 고수준 API(pipeline) 방식

모델과 토크나이저를 직접 사용할 수 있지만 pipeline 함수를 사용하는 것이 훨씬 쉽습니다.

In [ ]:
from transformers import pipeline

# 파이프라인을 만듭니다.
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=500,
    do_sample=False
)

- 💡 **[참고] PyTorch의 언어 모델 생성 함수에서 사용되는 주요 매개변수**

| 매개변수 | 의미 | 기본값 | 범위 | 설명 |
|--- |--- |--- |--- |--- |
| **max_new_tokens**  |모델이 생성할 새로운 토큰의 최대 개수  |모델마다 다름 (보통 50-100)  |1 ~ 모델의 최대 컨텍스트 길이  |  |
|**temperature**  |모델 출력의 창의성과 일관성을 조절하는 매개변수  |1.0  |0.0 ~ 2.0 (일반적으로 0.1 ~ 1.5)  |Softmax 함수의 온도 매개변수  |
|**do_sample**  |샘플링 방식 선택 (확률적 vs 결정적)  |False  |Boolean (True/False)  |  |

In [ ]:
#--------------------------
# max_new_tokens
#--------------------------
# 짧은 답변 (요약, 분류)
max_new_tokens=50

# 중간 길이 답변 (설명, Q&A)
max_new_tokens=200

# 긴 답변 (에세이, 보고서)
max_new_tokens=500

- **Temperature** 값에 따른 특성

|Temperature|특성|사용 상황|예시|
|--- |--- |--- |---|
|0.0 | 완전히 결정적, 항상 같은 출력 | 정확한 답변 필요 | 수학 문제, 번역 |
|0.3-0.5 | 안정적, 일관성 높음 | 비즈니스 문서, 기술 설명 | 보고서 작성 |
|0.7 | 균형 잡힌 창의성과 일관성 | 일반적인 대화, 교육 | 챗봇, Q&A |
|1.0 | 표준 창의성 | 기본 설정 | 기본 대화
|1.5-2.0 | 매우 창의적, 예측 불가 | 창작 활동, 브레인스토밍 | 소설, 시 창작 |

In [ ]:
#--------------------------
# Temperature 작동 원리
#--------------------------
# 수학적 표현
import torch.nn.functional as F

def apply_temperature(logits, temperature):
    """Temperature scaling 적용"""
    if temperature == 0:
        return torch.argmax(logits, dim=-1)
    else:
        scaled_logits = logits / temperature
        probabilities = F.softmax(scaled_logits, dim=-1)
        return torch.multinomial(probabilities, 1)

# 예시: 다음 단어 예측 확률
original_probs = [0.6, 0.3, 0.1]  # "좋은", "나쁜", "보통"

# Temperature = 0.5 (더 확실한 선택)
temp_05_probs = [0.8, 0.15, 0.05]

# Temperature = 1.5 (더 다양한 선택)
temp_15_probs = [0.45, 0.35, 0.2]

- **do_sample**옵션 비교

| do_sample |  방식 | 특징 | 언제 사용? |
|--- |--- |--- |--- |
| False | Greedy Decoding | 항상 가장 확률 높은 토큰 선택 | 정확성이 중요한 작업 |
| True | Sampling | 확률에 따라 랜덤하게 선택 | 창의성이 필요한 작업 |

In [ ]:
# do_sample=False (Greedy)
다음 단어 확률: {"좋은": 0.6, "나쁜": 0.3, "보통": 0.1}
선택: 항상 "좋은" (가장 높은 확률)

# do_sample=True (Sampling)
다음 단어 확률: {"좋은": 0.6, "나쁜": 0.3, "보통": 0.1}
선택: 60% 확률로 "좋은", 30% 확률로 "나쁜", 10% 확률로 "보통"

#### 프롬프트를 작성하고 모델에 주입

In [ ]:
# 프롬프트 (사용자 입력 / 쿼리)
messages = [
    {"role": "user", "content": "Create a funny joke about chickens."}
]

# 출력 생성
output = generator(messages)
print(output[0]["generated_text"])

### [실습] 기본 텍스트 생성

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

# 모델과 토크나이저 로드
model_name = "microsoft/Phi-3-mini-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # GPU 메모리 절약
    device_map="auto"
)

# 채팅 형식 프롬프트 구성
messages = [
    {"role": "system", "content": "당신은 도움이 되는 AI 어시스턴트입니다."},
    {"role": "user", "content": "파이썬에서 리스트와 튜플의 차이점을 설명해주세요."}
]

# 토큰화 및 생성
input_text = tokenizer.apply_chat_template(messages, tokenize=False)
inputs = tokenizer(input_text, return_tensors="pt")

# 텍스트 생성
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,  #모델이 생성할 새로운 토큰의 최대 개수
        temperature=0.7,
        do_sample=True
    )

# 결과 출력
print('-' * 50)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

### [실습] 코딩 어시스턴트

In [ ]:
def coding_assistant_example():
    messages = [
        {"role": "system", "content": "당신은 파이썬 프로그래밍 전문가입니다."},
        {"role": "user", "content": "피보나치 수열을 계산하는 효율적인 함수를 작성해주세요."}
    ]

    # 파이프라인 생성
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=300,
        temperature=0.3
    )

    # 응답 생성
    print('-' * 50)
    response = pipe(messages)
    print(response[0]['generated_text'])

coding_assistant_example()

### [실습] 교육용 Q&A 시스템

In [ ]:
def educational_qa_system():
    """교육용 질의응답 시스템"""

    class Phi3EducationBot:
        def __init__(self, model_name="microsoft/Phi-3-mini-4k-instruct"):
            self.tokenizer = AutoTokenizer.from_pretrained(model_name)
            self.model = AutoModelForCausalLM.from_pretrained(model_name)
            self.pipe = pipeline("text-generation",
                               model=self.model,
                               tokenizer=self.tokenizer)

        def answer_question(self, question, subject="컴퓨터공학"):
            messages = [
                {"role": "system", "content": f"당신은 {subject} 전문 강사입니다. 학생들이 이해하기 쉽게 설명해주세요."},
                {"role": "user", "content": question}
            ]

            response = self.pipe(messages, max_new_tokens=250)
            return response[0]['generated_text']

    # 사용 예시
    bot = Phi3EducationBot()
    answer = bot.answer_question("머신러닝과 딥러닝의 차이점은 무엇인가요?")
    print(answer)

educational_qa_system()

### [미션] OOO 시스템 만들기



---

